# Tensorflow GPU MNIST Model with GKE

**Please note: This tutorial uses Tensorflow-gpu=1.13.1, CUDA 10.0 and cuDNN 7.6**

**Requirements: Ubuntu 18.+ and Python 3.6**

In this tutorial we will run a deep MNIST Tensorflow example with GPU.

The tutorial will be broken down into the following sections:

1. Install all dependencies to run Tensorflow-GPU
    
    1.1 Installing CUDA 10.0
    
    1.2 Installing cuDNN 7.6
    
    1.3 Configure CUDA and cuDNN
    
    1.4 Install Tensorflow GPU
    
    
2. Train the MNIST model locally


3. Push the Image to your proejcts Container Registry


4. Deploy the model on GKE using Seldon Core


## Local Testing Environment

For the development of this example a GCE Virtual Machine was used to allow access to a GPU. The configuration for this VM is as follows:

* VM Image: TensorFlow from NVIDIA
* 8 vCPUs
* 32 GB memory
* 1x NVIDIA Tesla V100 GPU


## 1) Installing all dependencies to run Tensorflow-GPU

* Dependencies installed in this section:
    * Nvidia compute 3.0 onwards
    * CUDA 10.0
    * cuDNN 7.6
    * tensorflow-gpu 1.13.1

**Check Nvidia drivers >= 3.0**

In [ ]:
!nvidia-smi

## 1.1) Install CUDA 10.0

* **Download the CUDA 10.0 runfile**

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/10.0/Prod/local_installers/cuda_10.0.130_410.48_linux

* **Unpack the separate files:**

In [ ]:
! chmod +x cuda_10.0.130_410.48_linux
! ./cuda_10.0.130_410.48_linux --extract=$HOME

* **Install the Cuda 10.0 Toolkit file**:

From the terminal, run the following command
```
$ sudo ./cuda-linux.10.0.130-24817639.run
```
Hold 'd' to scroll to the bottom of the license agreement.

Accept the licencing agreement and all of the default settings.

* **Verify the install, by installing the sample test:**
```
$ sudo ./cuda-samples.10.0.130-24817639-linux.run
```
Again, accept the agreement and all default settings

* **Configure the runtime library:**

```
$ sudo bash -c "echo /usr/local/cuda/lib64/ > /etc/ld.so.conf.d/cuda.conf"
```

```
$ sudo ldconfig
```

* **Add the cuda bin to the file system:**

```
$ sudo vim /etc/environment
```

Add ‘:/usr/local/cuda/bin’ to the end of the PATH (inside quotes)

* **Reboot the system**

In [ ]:
!sudo shutdown -r now

* **Run the tests that we set up** - this takes some time to complete, so let it run for a little while...

```
$ cd /usr/local/cuda-10.0/samples

$ sudo make
```

If run into an error involving the GCC version:

```
$ sudo update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-6 10
```

```
$ sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-6 10
```

And run again, otherwise, skip this step.

* After complete, **run a devicequery and bandwidth test**:

In [ ]:
%%bash

cd /usr/local/cuda/samples/bin/x86_64/linux/release
./deviceQuery

**Remember to clean up by removing all of the downloaded runtime packages**

## 1.2) Install cuDNN 7.6

* **Download all 3 .deb files for CUDA10.0 and Ubuntu 18.04**

You will have to create a Nvidia account for this and go to the archive section of the cuDNN downloads

Ensure you download all 3 files:
- Runtime
- Developer
- Code Samples


**Unpackage the three files in this order**

In [ ]:
%%bash 
sudo dpkg -i ~/libcudnn7_7.6.0.64-1+cuda10.0_amd64.deb
sudo dpkg -i ~/libcudnn7-dev_7.6.0.64-1+cuda10.0_amd64.deb
sudo dpkg -i ~/libcudnn7-doc_7.6.0.64-1+cuda10.0_amd64.deb

* **Verify the install is successful with the MNIST example**

From the download folder. Copy the files to somewhere with write access: 

In [ ]:
! cp -r /usr/src/cudnn_samples_v7/ ~

**Go to the MNIST example code, compile and run it**

In [ ]:
%%bash
cd ~/cudnn_samples_v7/mnistCUDNN
sudo make
sudo ./mnistCUDNN

**Remember to clean up by removing all of the downloaded runtime packages**

## 1.3) Configure CUDA and cuDNN

**Add LD_LIBRARY_PATH in your .bashrc file:**

Add the following line in the end or your .bashrc file export export:

```
LD_LIBRARY_PATH=/usr/local/cuda/lib64:${LD_LIBRARY_PATH:+:${LD_LIBRARY_PATH}}
```

And source it with:

```
$ source ~/.bashrc
```

## 1.4) Install tensorflow with GPU

**Require v=1.13.1 as with CUDA 10.0**

In [ ]:
! pip3 install --upgrade tensorflow-gpu==1.13.1

In [ ]:
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

## 2) Train the MNIST model locally

* Wrap a Tensorflow MNIST python model for use as a prediction microservice in seldon-core
 
   * Run locally on Docker to test
   * Deploy on seldon-core running on minikube
 
## Dependencies

 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)

```bash
pip3 install seldon-core
```

## Train locally
 

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)
import tensorflow as tf

if __name__ == '__main__':
    
    x = tf.placeholder(tf.float32, [None,784], name="x")

    W = tf.Variable(tf.zeros([784,10]))
    b = tf.Variable(tf.zeros([10]))

    y = tf.nn.softmax(tf.matmul(x,W) + b, name="y")

    y_ = tf.placeholder(tf.float32, [None, 10])


    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

    init = tf.initialize_all_variables()

    sess = tf.Session()
    sess.run(init)

    for i in range(1000):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print(sess.run(accuracy, feed_dict = {x: mnist.test.images, y_:mnist.test.labels}))

    saver = tf.train.Saver()

    saver.save(sess, "model/deep_mnist_model")



Wrap model using s2i

In [ ]:
!s2i build . seldonio/seldon-core-s2i-python3-tf-gpu:0.1 deep-mnist-gpu:0.1

In [ ]:
!docker run --name "mnist_predictor" -d --rm -p 5000:5000 deep-mnist-gpu:0.1

Send some random features that conform to the contract

In [ ]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p

In [ ]:
!docker rm mnist_predictor --force

## 3) Push the image to Google Container Registry

**Configure access to container registry** (follow the configuration to link to your own project).

```
$ gcloud auth configure-docker
```

**Tag Image with your project's registry path** (Edit the command below)

In [ ]:
!docker tag deep-mnist-gpu:0.1 gcr.io/<YOUR_PROJECT_ID>/deep-mnist-gpu:0.1

**Push the Image to the Container Registry** (Again edit command below)

In [ ]:
!docker push gcr.io/<YOUR_PROJECT_ID>/deep-mnist-gpu:0.1

## 4) Deploy in GKE

## Spin up a GKE Cluster

For this example only one node is needed within the cluster. The cluster should have the following **config**:

* 8 CPUs
* 30 GB Total Memory
* 1 Node with 1X NVIDIA Tesla V100 GPU
* Ubuntu Node image

Leave the rest of the config as default. 

**Connect to your cluster and check the context.**

In [ ]:
!gcloud config set project <YOUR_PROJECT_ID>
!gcloud container clusters get-credentials <YOUR_CLUSTER_NAME>
!kubectl config current-context

**Installing NVIDIA GPU device drivers**

(The below command is for the Ubuntu Node Image - if using a COS image, please see the Google Cloud Documentation for the correct command).

In [ ]:
!kubectl apply -f https://raw.githubusercontent.com/GoogleCloudPlatform/container-engine-accelerators/master/nvidia-driver-installer/ubuntu/daemonset-preloaded.yaml

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Setup-Cluster) with [Ambassador Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Ambassador) and [Install Seldon Core](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Install-Seldon-Core). Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

## Build the Seldon Graph

First lets look at the Seldon Graph Yaml file:

In [74]:
!cat deep_mnist_gpu.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "deep-mnist-gpu"
    },
    "spec": {
        "annotations": {
            "project_name": "Tensorflow MNIST",
            "deployment_version": "v1"
        },
        "name": "deep-mnist-gpu",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "gcr.io/<YOUR_PROJECT_ID>/deep-mnist-gpu:0.1",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
				    "limits": {
					"nvidia.com/gpu": 1
                                    }
				}
                            }
                        ],
                        "terminationGracePeri

**Change the image name in this file (line 24) to match the path to the image in your container registry.**

```
$vim deep_mnist_gpu.json
```

Next, we are ready to **build the seldon graph**.

In [65]:
!kubectl create -f deep_mnist_gpu.json

seldondeployment.machinelearning.seldon.io/deep-mnist-gpu created


In [15]:
!kubectl rollout status deploy/deep-mnist-gpu-single-model-8969cc0

Error from server (NotFound): deployments.extensions "deep-mnist-gpu-single-model-8969cc0" not found


Check the deployment is running

In [71]:
!kubectl get pods

NAME                                                   READY   STATUS    RESTARTS   AGE
ambassador-865c877494-2td9s                            1/1     Running   0          101m
ambassador-865c877494-2vsk2                            1/1     Running   0          101m
ambassador-865c877494-qzh4c                            1/1     Running   0          101m
deep-mnist-gpu-single-model-0588ac2-865d745b7d-kqcp9   2/2     Running   0          71m
seldon-operator-controller-manager-0                   1/1     Running   1          101m


## Test the deployment with test data

**Change the IP address to the External IP of your Ambassador deployment.**

In [72]:
!kubectl get svc

NAME                                         TYPE           CLUSTER-IP     EXTERNAL-IP     PORT(S)                      AGE
ambassador                                   LoadBalancer   10.76.8.138    104.197.71.69   80:30783/TCP,443:32277/TCP   101m
ambassador-admins                            ClusterIP      10.76.12.144   <none>          8877/TCP                     101m
deep-mnist-gpu-deep-mnist-gpu                ClusterIP      10.76.5.205    <none>          8000/TCP,5001/TCP            71m
kubernetes                                   ClusterIP      10.76.0.1      <none>          443/TCP                      107m
seldon-87fe3957f4554e9b5af993717a0b9327      ClusterIP      10.76.14.160   <none>          9000/TCP                     71m
seldon-operator-controller-manager-service   ClusterIP      10.76.8.100    <none>          443/TCP                      101m
webhook-server-service                       ClusterIP      10.76.7.151    <none>          443/TCP                      101m


In [73]:
!seldon-core-api-tester contract.json <EXTERNAL_IP_ADDRESS> `kubectl get svc ambassador -o jsonpath='{.spec.ports[0].port}'` \
    deep-mnist-gpu --namespace default -p

----------------------------------------
SENDING NEW REQUEST:

[[0.798 0.827 0.034 0.384 0.938 0.036 0.135 0.555 0.86  0.263 0.411 0.894
  0.327 0.865 0.906 0.914 0.133 0.565 0.803 0.417 0.825 0.678 0.805 0.206
  0.017 0.698 0.41  0.503 0.984 0.214 0.468 0.366 0.132 0.973 0.472 0.346
  0.001 0.662 0.412 0.537 0.522 0.242 0.289 0.676 0.379 0.542 0.452 0.467
  0.392 1.    0.771 0.442 0.352 0.505 0.259 0.505 0.664 0.942 0.457 0.417
  0.895 0.42  0.322 0.885 0.578 0.528 0.222 0.283 0.137 0.605 0.915 0.182
  0.42  0.94  0.262 0.599 0.552 0.437 0.179 0.928 0.831 0.193 0.391 0.416
  0.315 0.012 0.815 0.925 0.52  0.773 0.93  0.673 0.757 0.979 0.151 0.459
  0.621 0.553 0.605 0.176 0.702 0.814 0.784 0.952 0.513 0.125 0.68  0.043
  0.377 0.67  0.466 0.824 0.245 0.221 0.324 0.749 0.182 0.992 0.243 0.855
  0.477 0.176 0.262 0.537 0.69  0.717 0.059 0.711 0.26  0.149 0.34  0.71
  0.041 0.623 0.447 0.319 0.089 0.954 0.435 0.267 0.416 0.275 0.923 0.254
  0.542 0.995 0.782 0.337 0.991 0.187 0.183 0.479 

## Clean up

Make sure you delete the cluster once you have finished with it to avoid any ongoing charges.

In [ ]:
!gcloud container clusters delete <YOUR_CLUSTER_NAME>